In [7]:
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["GEMINI_API_KEY"] = ""

In [8]:
import os
import requests
import json
import time
import statistics
from dataclasses import dataclass, field
from typing import Any, List, Optional, TypedDict, Dict, Callable
from contextlib import contextmanager
from functools import wraps
from langchain_core.prompts import PromptTemplate
from langchain_core.language_models.llms import BaseLLM
from langchain_core.outputs import LLMResult
from pydantic import Field
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

## Instrumentation Module

This module provides comprehensive latency and throughput measurement for the LangGraph agent.

In [9]:
# --- INSTRUMENTATION CODE ---

@dataclass
class TimingRecord:
    """Single timing measurement record."""
    name: str
    start_time: float
    end_time: float
    duration_ms: float
    metadata: Dict[str, Any] = field(default_factory=dict)


@dataclass
class LLMCallMetrics:
    """Metrics for a single LLM API call."""
    latency_ms: float
    input_chars: int
    output_chars: int
    model_name: str
    success: bool
    error_message: Optional[str] = None


class InstrumentationCollector:
    """Collects and aggregates timing/throughput metrics across the pipeline."""

    def __init__(self):
        self.timing_records: List[TimingRecord] = []
        self.llm_metrics: List[LLMCallMetrics] = []
        self.node_timings: Dict[str, List[float]] = {}
        self.graph_start_time: Optional[float] = None
        self.graph_end_time: Optional[float] = None
        self._run_count: int = 0

    def reset(self):
        """Reset all collected metrics for a new run."""
        self.timing_records = []
        self.llm_metrics = []
        self.node_timings = {}
        self.graph_start_time = None
        self.graph_end_time = None

    def start_graph(self):
        """Mark the start of graph execution."""
        self.graph_start_time = time.perf_counter()
        self._run_count += 1

    def end_graph(self):
        """Mark the end of graph execution."""
        self.graph_end_time = time.perf_counter()

    @contextmanager
    def measure(self, name: str, **metadata):
        """Context manager to measure execution time of a code block."""
        start = time.perf_counter()
        try:
            yield
        finally:
            end = time.perf_counter()
            duration_ms = (end - start) * 1000
            record = TimingRecord(
                name=name,
                start_time=start,
                end_time=end,
                duration_ms=duration_ms,
                metadata=metadata
            )
            self.timing_records.append(record)

            # Also track by node name for aggregation
            if name not in self.node_timings:
                self.node_timings[name] = []
            self.node_timings[name].append(duration_ms)

    def record_llm_call(self, metrics: LLMCallMetrics):
        """Record metrics from an LLM API call."""
        self.llm_metrics.append(metrics)

    def get_total_latency_ms(self) -> float:
        """Get total graph execution latency in milliseconds."""
        if self.graph_start_time and self.graph_end_time:
            return (self.graph_end_time - self.graph_start_time) * 1000
        return 0.0

    def get_llm_latency_stats(self) -> Dict[str, float]:
        """Get statistics on LLM call latencies."""
        if not self.llm_metrics:
            return {}

        latencies = [m.latency_ms for m in self.llm_metrics]
        return {
            "count": len(latencies),
            "total_ms": sum(latencies),
            "mean_ms": statistics.mean(latencies),
            "min_ms": min(latencies),
            "max_ms": max(latencies),
            "median_ms": statistics.median(latencies),
            "stdev_ms": statistics.stdev(latencies) if len(latencies) > 1 else 0.0
        }

    def get_throughput_stats(self) -> Dict[str, float]:
        """Calculate throughput metrics."""
        total_time_sec = self.get_total_latency_ms() / 1000
        if total_time_sec == 0:
            return {}

        total_input_chars = sum(m.input_chars for m in self.llm_metrics)
        total_output_chars = sum(m.output_chars for m in self.llm_metrics)
        num_requests = len(self.llm_metrics)

        # Estimate tokens (rough: ~4 chars per token)
        est_input_tokens = total_input_chars / 4
        est_output_tokens = total_output_chars / 4

        return {
            "requests_per_second": num_requests / total_time_sec,
            "input_chars_per_second": total_input_chars / total_time_sec,
            "output_chars_per_second": total_output_chars / total_time_sec,
            "est_input_tokens_per_second": est_input_tokens / total_time_sec,
            "est_output_tokens_per_second": est_output_tokens / total_time_sec,
            "total_input_chars": total_input_chars,
            "total_output_chars": total_output_chars,
            "est_total_input_tokens": int(est_input_tokens),
            "est_total_output_tokens": int(est_output_tokens)
        }

    def get_node_breakdown(self) -> Dict[str, Dict[str, float]]:
        """Get timing breakdown by node."""
        breakdown = {}
        for name, timings in self.node_timings.items():
            breakdown[name] = {
                "count": len(timings),
                "total_ms": sum(timings),
                "mean_ms": statistics.mean(timings),
                "min_ms": min(timings),
                "max_ms": max(timings)
            }
        return breakdown

    def print_report(self):
        """Print a formatted instrumentation report."""
        print("\n" + "="*60)
        print("INSTRUMENTATION REPORT")
        print("="*60)

        # Total Latency
        print(f"\n📊 TOTAL LATENCY")
        print(f"   Total Graph Execution: {self.get_total_latency_ms():.2f} ms")

        # Node Breakdown
        print(f"\n📍 NODE-LEVEL BREAKDOWN")
        breakdown = self.get_node_breakdown()
        for node_name, stats in breakdown.items():
            print(f"   {node_name}:")
            print(f"      - Total: {stats['total_ms']:.2f} ms")
            print(f"      - Mean:  {stats['mean_ms']:.2f} ms")

        # LLM Call Statistics
        print(f"\n🤖 LLM CALL STATISTICS")
        llm_stats = self.get_llm_latency_stats()
        if llm_stats:
            print(f"   Number of calls: {llm_stats['count']}")
            print(f"   Total LLM time:  {llm_stats['total_ms']:.2f} ms")
            print(f"   Mean latency:    {llm_stats['mean_ms']:.2f} ms")
            print(f"   Min latency:     {llm_stats['min_ms']:.2f} ms")
            print(f"   Max latency:     {llm_stats['max_ms']:.2f} ms")
            print(f"   Median latency:  {llm_stats['median_ms']:.2f} ms")

        # Throughput
        print(f"\n🚀 THROUGHPUT METRICS")
        throughput = self.get_throughput_stats()
        if throughput:
            print(f"   Requests/second:       {throughput['requests_per_second']:.4f}")
            print(f"   Output chars/second:   {throughput['output_chars_per_second']:.2f}")
            print(f"   Est. output tokens/s:  {throughput['est_output_tokens_per_second']:.2f}")
            print(f"   Total input chars:     {throughput['total_input_chars']}")
            print(f"   Total output chars:    {throughput['total_output_chars']}")

        # Detailed Timeline
        print(f"\n⏱️  EXECUTION TIMELINE")
        for record in self.timing_records:
            print(f"   [{record.duration_ms:8.2f} ms] {record.name}")

        print("\n" + "="*60)

    def to_dict(self) -> Dict[str, Any]:
        """Export all metrics as a dictionary."""
        return {
            "total_latency_ms": self.get_total_latency_ms(),
            "llm_statistics": self.get_llm_latency_stats(),
            "throughput": self.get_throughput_stats(),
            "node_breakdown": self.get_node_breakdown(),
            "run_count": self._run_count,
            "timeline": [
                {"name": r.name, "duration_ms": r.duration_ms, "metadata": r.metadata}
                for r in self.timing_records
            ]
        }


# Global collector instance
collector = InstrumentationCollector()


def instrumented_node(name: str):
    """Decorator to instrument a LangGraph node function."""
    def decorator(func: Callable):
        @wraps(func)
        def wrapper(state, *args, **kwargs):
            with collector.measure(name):
                return func(state, *args, **kwargs)
        return wrapper
    return decorator

In [10]:
# --- 1. Custom LLM Implementation (with Instrumentation) ---

class CustomHTTPGemini(BaseLLM):
    """
    A custom LangChain LLM wrapper that interacts with the Google Gemini API
    using direct HTTP requests, with built-in instrumentation.
    """

    api_key: Optional[str] = None
    model_name: str = Field(default="gemini-2.5-flash", alias="model")
    base_url: str = "https://generativelanguage.googleapis.com/v1beta/models/"
    response_schema: Optional[dict] = None

    def __init__(self, **kwargs: Any):
        super().__init__(**kwargs)
        if not self.api_key:
            self.api_key = os.getenv("GEMINI_API_KEY")
        if not self.api_key:
            raise ValueError("GEMINI_API_KEY must be provided or set as an environment variable.")

    @property
    def _llm_type(self) -> str:
        return "custom_http_gemini"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> str:
        """
        Make the HTTP POST request to the Gemini API with instrumentation.
        """
        api_endpoint = f"{self.base_url}{self.model_name}:generateContent"
        url = f"{api_endpoint}?key={self.api_key}"
        headers = {"Content-Type": "application/json"}

        request_data = {
            "contents": [{"parts": [{"text": prompt}]}]
        }

        if self.response_schema:
            request_data["generationConfig"] = {
                "responseMimeType": "application/json",
                "responseSchema": self.response_schema
            }

        # --- INSTRUMENTATION: Measure LLM call ---
        input_chars = len(prompt)
        start_time = time.perf_counter()

        try:
            response = requests.post(url=url, headers=headers, json=request_data)
            response.raise_for_status()
            response_json = response.json()
            generated_text = response_json['candidates'][0]['content']['parts'][0]['text']

            end_time = time.perf_counter()
            latency_ms = (end_time - start_time) * 1000

            # Record metrics
            collector.record_llm_call(LLMCallMetrics(
                latency_ms=latency_ms,
                input_chars=input_chars,
                output_chars=len(generated_text),
                model_name=self.model_name,
                success=True
            ))

            return generated_text

        except requests.exceptions.HTTPError as err:
            end_time = time.perf_counter()
            latency_ms = (end_time - start_time) * 1000

            # Record failed call
            collector.record_llm_call(LLMCallMetrics(
                latency_ms=latency_ms,
                input_chars=input_chars,
                output_chars=0,
                model_name=self.model_name,
                success=False,
                error_message=str(err)
            ))

            error_message = f"Gemini API HTTP Error ({err.response.status_code}): {err.response.text}"
            raise RuntimeError(error_message) from err
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred during API call: {e}")

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> LLMResult:
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop, run_manager, **kwargs)
            generations.append([{"text": text}])
        return LLMResult(generations=generations)

In [11]:
# --- 2. LangGraph AGENT DEFINITIONS (with Instrumentation) ---

# 2.1. Define the State for the Graph
class AgentState(TypedDict):
    item_name: str
    complexity_level: str
    score: Optional[float]
    review_data: Optional[str]


# --- JSON Schema Definitions ---
SCORE_SCHEMA = {
    "type": "OBJECT",
    "properties": {
        "score": {"type": "NUMBER", "description": "A random technical score between 0.0 and 1.0."},
    },
    "required": ["score"],
    "propertyOrdering": ["score"]
}

REVIEW_SCHEMA = {
    "type": "OBJECT",
    "properties": {
        "review_text": {"type": "STRING", "description": "A concise, technical review."},
        "category": {"type": "STRING", "description": "The category of the review (e.g., 'Positive', 'Neutral', 'Negative')."}
    },
    "required": ["review_text", "category"],
    "propertyOrdering": ["review_text", "category"]
}


# 2.2. Node for Agent 1: Score Generator (INSTRUMENTED)
@instrumented_node("score_generator_node")
def score_generator_node(state: AgentState) -> dict:
    """Generates a technical score (0.0 to 1.0) for the item."""
    print("--- [Agent 1] Executing: Score Generator ---")

    llm_score_generator = CustomHTTPGemini(model_name="gemini-2.5-flash", response_schema=SCORE_SCHEMA)

    prompt_1 = PromptTemplate.from_template(
        "You are a technical analyst. Your task is to assign a random score between 0.0 and 1.0 to the '{item_name}' based on its complexity '{complexity_level}'. Output the result strictly in JSON format according to the schema."
    )
    prompt_value = prompt_1.format(
        item_name=state['item_name'],
        complexity_level=state['complexity_level']
    )

    raw_json_output = llm_score_generator.invoke(prompt_value)

    try:
        score_data = json.loads(raw_json_output)
        score = score_data.get('score', 0.0)
        print(f"--- [Agent 1] Generated Score: {score} ---")
        return {"score": score}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from Agent 1: {e}. Falling back to score 0.5.")
        return {"score": 0.5}


# 2.3. Node for Agent 2: Review Generator (INSTRUMENTED)
@instrumented_node("review_generator_node")
def review_generator_node(state: AgentState) -> dict:
    """Generates a detailed review based on the generated score and context."""
    print("\n--- [Agent 2] Executing: Review Generator ---")

    llm_review_generator = CustomHTTPGemini(model_name="gemini-2.5-flash", response_schema=REVIEW_SCHEMA)

    prompt_2 = PromptTemplate.from_template(
        "Generate a technical review for the item '{item_name}' which has a complexity of '{complexity_level}' and received a technical score of {score}. Your review must reflect this score. Output the review strictly in JSON format according to the schema."
    )

    prompt_value = prompt_2.format(
        item_name=state['item_name'],
        complexity_level=state['complexity_level'],
        score=state['score']
    )

    raw_json_output = llm_review_generator.invoke(prompt_value)

    print("--- [Agent 2] Generated Review Data ---")
    return {"review_data": raw_json_output}

In [12]:
# --- 2.4. LangGraph Setup and Execution (with Instrumentation) ---

if __name__ == "__main__":
    print("--- LangGraph Custom HTTP Gemini Cascading Agent Example ---")
    print("NOTE: With Instrumentation for Latency & Throughput Measurement")

    if not os.getenv("GEMINI_API_KEY"):
        print("\nERROR: GEMINI_API_KEY environment variable not set.")
        print("Please set the GEMINI_API_KEY environment variable and try again.")
        exit()

    # --- Build the Graph ---
    graph_builder = StateGraph(AgentState)
    graph_builder.add_node("score_generator", score_generator_node)
    graph_builder.add_node("review_generator", review_generator_node)
    graph_builder.set_entry_point("score_generator")
    graph_builder.add_edge("score_generator", "review_generator")
    graph_builder.add_edge("review_generator", END)
    app = graph_builder.compile()

    # --- Example Invocation with Instrumentation ---
    print("\nInvoking the two-agent LangGraph application...")

    initial_state = {
        "item_name": "Quantum Entanglement Module v1.2",
        "complexity_level": "High/Experimental"
    }

    # Reset collector and start timing
    collector.reset()
    collector.start_graph()

    # Run the graph
    final_state = app.invoke(initial_state)

    # End timing
    collector.end_graph()

    final_review_json = final_state.get('review_data')
    final_score = final_state.get('score')

    print("\n--- Graph Execution Complete ---")
    print(f"Original Input:\n{json.dumps(initial_state, indent=2)}")
    print(f"Intermediate Score Generated by Agent 1: {final_score}")

    print("\n--- Final Output (Generated by Agent 2) ---")
    try:
        print(json.dumps(json.loads(final_review_json), indent=2))
    except json.JSONDecodeError:
        print(f"Raw Output:\n{final_review_json}")

    # --- Print Instrumentation Report ---
    collector.print_report()

--- LangGraph Custom HTTP Gemini Cascading Agent Example ---
NOTE: With Instrumentation for Latency & Throughput Measurement

Invoking the two-agent LangGraph application...
--- [Agent 1] Executing: Score Generator ---
--- [Agent 1] Generated Score: 0.73 ---

--- [Agent 2] Executing: Review Generator ---
--- [Agent 2] Generated Review Data ---

--- Graph Execution Complete ---
Original Input:
{
  "item_name": "Quantum Entanglement Module v1.2",
  "complexity_level": "High/Experimental"
}
Intermediate Score Generated by Agent 1: 0.73

--- Final Output (Generated by Agent 2) ---
{
  "review_text": "The Quantum Entanglement Module v1.2, despite its High/Experimental classification, demonstrates promising capabilities in establishing stable entangled states. Achieving an entanglement fidelity of 0.73, it offers consistent operation, though further optimization in decoherence mitigation techniques would enhance its performance. Its architecture allows for relatively low-latency entanglement

## Batch Throughput Testing

Run multiple iterations to get more accurate throughput measurements.

In [13]:
def run_batch_throughput_test(app, test_cases: List[Dict], num_iterations: int = 5):
    """
    Run multiple iterations to measure aggregate throughput.

    Args:
        app: Compiled LangGraph application
        test_cases: List of initial states to test
        num_iterations: Number of times to run each test case
    """
    all_metrics = []

    print(f"\n{'='*60}")
    print(f"BATCH THROUGHPUT TEST")
    print(f"Running {len(test_cases)} test cases x {num_iterations} iterations")
    print(f"{'='*60}")

    total_start = time.perf_counter()

    for i, test_case in enumerate(test_cases):
        print(f"\nTest case {i+1}: {test_case.get('item_name', 'Unknown')}")

        for iteration in range(num_iterations):
            collector.reset()
            collector.start_graph()

            try:
                _ = app.invoke(test_case)
            except Exception as e:
                print(f"  Iteration {iteration+1}: ERROR - {e}")
                continue

            collector.end_graph()

            metrics = collector.to_dict()
            metrics['test_case_index'] = i
            metrics['iteration'] = iteration
            all_metrics.append(metrics)

            print(f"  Iteration {iteration+1}: {metrics['total_latency_ms']:.2f} ms")

    total_end = time.perf_counter()
    total_time_sec = total_end - total_start

    # Aggregate statistics
    print(f"\n{'='*60}")
    print("AGGREGATE RESULTS")
    print(f"{'='*60}")

    latencies = [m['total_latency_ms'] for m in all_metrics]
    llm_latencies = []
    for m in all_metrics:
        if m['llm_statistics']:
            llm_latencies.append(m['llm_statistics'].get('mean_ms', 0))

    successful_runs = len(all_metrics)
    total_requests = sum(m['llm_statistics'].get('count', 0) for m in all_metrics if m['llm_statistics'])

    print(f"\nTotal runs completed: {successful_runs}")
    print(f"Total wall-clock time: {total_time_sec:.2f} seconds")
    print(f"Total LLM requests: {total_requests}")
    print(f"\nEnd-to-end Latency (per run):")
    print(f"  Mean:   {statistics.mean(latencies):.2f} ms")
    print(f"  Median: {statistics.median(latencies):.2f} ms")
    print(f"  Min:    {min(latencies):.2f} ms")
    print(f"  Max:    {max(latencies):.2f} ms")
    if len(latencies) > 1:
        print(f"  Stdev:  {statistics.stdev(latencies):.2f} ms")

    if llm_latencies:
        print(f"\nLLM Call Latency (mean per run):")
        print(f"  Mean:   {statistics.mean(llm_latencies):.2f} ms")
        print(f"  Median: {statistics.median(llm_latencies):.2f} ms")

    print(f"\nThroughput:")
    print(f"  Runs/second:     {successful_runs / total_time_sec:.4f}")
    print(f"  Requests/second: {total_requests / total_time_sec:.4f}")

    return all_metrics

In [17]:


test_cases = [
    {"item_name": "Quantum Entanglement Module v1.2", "complexity_level": "High/Experimental"},
    {"item_name": "Basic Data Logger", "complexity_level": "Low/Simple"},
    {"item_name": "ML Inference Pipeline", "complexity_level": "Medium/Standard"},
]

batch_results = run_batch_throughput_test(app, test_cases, num_iterations=3)


BATCH THROUGHPUT TEST
Running 3 test cases x 3 iterations

Test case 1: Quantum Entanglement Module v1.2
--- [Agent 1] Executing: Score Generator ---
--- [Agent 1] Generated Score: 0.76 ---

--- [Agent 2] Executing: Review Generator ---
--- [Agent 2] Generated Review Data ---
  Iteration 1: 4708.23 ms
--- [Agent 1] Executing: Score Generator ---
--- [Agent 1] Generated Score: 0.7324 ---

--- [Agent 2] Executing: Review Generator ---
--- [Agent 2] Generated Review Data ---
  Iteration 2: 4391.81 ms
--- [Agent 1] Executing: Score Generator ---
  Iteration 3: ERROR - Gemini API HTTP Error (429): {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests,

## Export Metrics to JSON

Save metrics for external analysis or dashboards.

In [15]:
# Export metrics to JSON file
def export_metrics_to_json(filename: str = "metrics.json"):
    """Export collected metrics to a JSON file."""
    metrics = collector.to_dict()
    with open(filename, 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"Metrics exported to {filename}")

# Uncomment to export
# export_metrics_to_json("langgraph_metrics.json")